# 要約 
このJupyterノートブックは、「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」において、ユーザーの応答選好を予測するためのモデルを開発することを目的としています。特に、事前トレーニング済みのLLM（大規模言語モデル）をファインチューニングするために、LoRA（Low-Rank Adaptation）を用いた手法が採用されています。

## 取り組んでいる問題
このノートブックは、Chatbot Arenaのデータセットを利用して、ユーザーがどちらのチャットボットの応答を好むかを予測するモデルの構築に焦点を当てています。データセットには、さまざまなプロンプトに対応する複数の応答が含まれており、ユーザーの好みを学習し予測することが目的です。

## 使用している手法
主な手法として、以下の技術が使用されています：
1. **LoRA**: モデルのファインチューニングのために用いられる低ランク適応手法。これにより、パラメータ数を削減しつつモデルの性能を維持し、効率的なトレーニングが可能になります。
2. **量子化**: モデルを4ビットで読み込み、メモリ使用量を削減し、計算効率を向上させるために`BitsAndBytesConfig`を使用しています。
3. **データ前処理**: ユーザープロンプトと応答を結合し、トークナイズ（テキストデータを数値に変換する処理）を行います。この処理にはHugging Faceの`transformers`ライブラリが利用されています。

## 使用しているライブラリ
- **Pandas**: データの操作と分析。
- **PyTorch**: モデルのトレーニングとデータローディングに使用。
- **Transformers**: 事前トレーニング済みモデルやトークナイザーの取得。
- **PEFT (Parameter Efficient Fine-Tuning)**: LoRAを使用するための機能。
- **Datasets**: データセットの管理と前処理を簡素化。
- **TRL (Transformers Reinforcement Learning)**: モデルのトレーニングとフィードバックのためのフレームワーク。

約1エポックでモデルをトレーニングし、エポックごとに評価を行いながら、最良のモデルを保存する仕組みが組まれています。

このノートブックは、最終的には、ユーザーが好む応答を予測する効果的なモデルを構築することを目指しています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に基づき、機械学習・深層学習の初心者が特につまずきそうな専門用語やコンセプトについての簡単な解説です。

1. **bitsandbytes**:
   - **説明**: モデルのストレージや計算効率を向上させるためのライブラリで、主に量子化や低精度計算をサポートします。このノートブックでは、モデルサイズを小さく保ちながら高速計算を実現するために使用されています。

2. **LoraConfig**:
   - **説明**: LoRA（Low-Rank Adaptation）は、モデルを特定のタスクに少ないパラメータで適応させるための方法です。LoRAでは、特定の層の重みを低ランク行列で修正し、少ない計算資源で効率的な調整を行います。

3. **量子化 (Quantization)**:
   - **説明**: モデルの数値精度を落とし、演算に必要なビット数を減らす技術です。例えば、32ビットの浮動小数点数ではなく、4ビットや8ビットの整数を使用することでモデルサイズを縮小し、推論速度を向上させます。

4. **バイアス (Bias)**:
   - **説明**: 機械学習モデルにおいて、モデルが訓練データに対して持つ先入観や偏りを指します。いかにして学習の過程においてバイアスを設定するかが、モデル性能の向上に影響します。

5. **パディング (Padding)**:
   - **説明**: シーケンスデータを一定の長さに揃えるために、短いシーケンスの末尾に特別なトークン（パディングトークン）を追加するプロセスです。これによって、バッチ処理において形状を揃えることが可能になります。

6. **EarlyStoppingCallback**:
   - **説明**: モデルの訓練中に、一定の評価基準（たとえばバリデーション損失）が改善しなくなった場合に訓練を早期に停止するための機能です。過学習を防ぐために重要です。

7. **デバイスマップ (Device Map)**:
   - **説明**: モデルの各部分をどのハードウェア（CPUやGPU）で実行するかを自動的に設定する方法です。大規模モデルを効率的に訓練する、または推論するために利用されます。

8. **タイポロジ (Topology)**:
   - **説明**: モデルの構造や配置を表現します。特にプリトレーニングの文脈では、モデルがどのようなアーキテクチャを持つのかや、トレーニング手法がどのように設定されるかを示します。

9. **データセット辞書 (DatasetDict)**:
   - **説明**: 複数のデータセット（ここでは訓練用と検証用）を1つのオブジェクトで管理するための構造です。PyTorchやTransformersライブラリでデータを簡単に扱うのに便利です。

これらは、特にJupyter Notebook内で使われる専門用語やコンセプトで、初心者が見逃しがちな部分かもしれません。

---


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer,
    EarlyStoppingCallback
)
from trl import SFTTrainer,setup_chat_format
import numpy as np
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoTokenizer
from datasets import DatasetDict, Dataset

In [ ]:
class CFG:
    VER = 1
    NUM_LABELS = 3
    BATCH_SIZE = 4
    EPOCHS = 1
    
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'

    
    SEED = 2024 
    MAX_LENGTH = 1024 
    
    OUTPUT_DIR = 'Llama 3 8b fine-tuned model'

# データセットの準備


In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
train = train[:int(len(train)*0.01)]

# 入力文字列を処理する関数
def process(input_str):
    stripped_str = input_str.strip('[]')  # 角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文を取り出し、前後の空白を取り除く
    return  ' '.join(sentences)  # 文を結合して返す

# プロンプトと応答を処理
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

# 'Null'の応答を削除して訓練データを整形
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)  # 'null'行を削除
train.reset_index(inplace=True, drop=True)  # インデックスをリセット
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)  # ラベルを設定

print(f"Total {len(indexes)} Null response rows dropped")  # 削除された行数を表示
print('Total train samples: ', len(train))  # 残りのサンプル数を表示

# 入力と応答をテキスト形式に変換
train['text'] = 'ユーザープロンプト: ' + train['prompt'] +  '\n\nモデルA :\n' + train['response_a'] +'\n\n--------\n\nモデルB:\n'  + train['response_b']
print(train['text'][4])  # 5番目のテキストを表示

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)  # データを訓練データと検証データに分割

dataset_train = Dataset.from_pandas(train_df)  # 訓練データセットの作成
dataset_val = Dataset.from_pandas(val_df)  # 検証データセットの作成

# データセットを1つのDatasetDictに結合
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val,
})

# モデルの読み込み - 量子化


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,  # 4ビットでモデルを読み込む設定
    bnb_4bit_quant_type = 'nf4',  # 量子化のタイプ
    bnb_4bit_use_double_quant = True,  # ダブル量子化を使用するか
    bnb_4bit_compute_dtype = torch.bfloat16  # 計算のデータ型
)

# 事前トレーニング済みモデルを読み込み
model = AutoModelForSequenceClassification.from_pretrained(
    CFG.MODEL_NAME,  # モデル名
    quantization_config=quantization_config,  # 量子化設定を適用
    num_labels=CFG.NUM_LABELS,  # ラベルの数
    device_map='auto'  # 自動でデバイスを設定
)

# LoRAの設定


In [ ]:
lora_config = LoraConfig(
    r = 16,  # LoRAのrパラメータ
    lora_alpha = 8,  # 構成パラメータ
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],  # ターゲットモジュールの指定
    lora_dropout = 0.05,  # ドロップアウト率
    bias = 'none',  # バイアス設定
    task_type = 'SEQ_CLS'  # タスクのタイプ
)

# モデルのkビットトレーニング用の準備
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)  # LoRAモデルを取得

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME, add_prefix_space=True)  # トークナイザーを読み込み

# パディングトークンの設定
tokenizer.pad_token_id = tokenizer.eos_token_id  # パディングトークンIDを設定
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを設定
tokenizer.padding_side = 'right'  # パディング側を右に設定
tokenizer.add_eos_token = True  # EOSトークンを追加

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id  # モデルにパディングトークンIDを設定
# model.config.use_cache = False  # キャッシュの使用を無効化する設定（コメントアウト）
model.config.pretraining_tp = 1  # プリトレーニングのタイポロジを設定

In [ ]:
# データの前処理を行う関数
def data_preprocesing(row):
    return tokenizer(row['text'], padding='max_length', truncation=True, max_length=CFG.MAX_LENGTH, return_tensors='np')  # テキストをトークナイズ

# トークナイズされたデータを生成
tokenized_data = dataset.map(data_preprocesing, batched=True, remove_columns=['text'])  # データセットにマッピング
tokenized_data.set_format("torch")  # PyTorch形式に設定

# Llama 3のファインチューニング: モデルのトレーニング


In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=CFG.BATCH_SIZE,  # デバイスごとの訓練バッチサイズ
    num_train_epochs=CFG.EPOCHS,  # 訓練エポック数
    logging_dir = f'./logs_v{CFG.VER}',  # ロギングディレクトリ
    output_dir = f'./output_v{CFG.VER}',  # 出力ディレクトリ
    logging_steps=10,  # ロギングのステップ数
    save_steps=10,  # モデルの保存のステップ数
    logging_first_step=True,  # 最初のステップのロギングを有効にする
    overwrite_output_dir=True,  # 出力ディレクトリの上書きを許可
    warmup_ratio=0.0,  # ウォームアップ比率
    learning_rate=5e-5,  # 学習率
    lr_scheduler_type='constant',  # 学習率スケジューラのタイプ
    weight_decay=0.01,  # 重みの減衰
    eval_steps=10,  # 評価のステップ数
    evaluation_strategy='steps',  # 評価戦略
    save_total_limit=2,  # 保存するトータル数の制限
    report_to='none',  # ロギングの報告先をなしに設定
    load_best_model_at_end = True  # 最後に最良モデルを読み込む
)

In [ ]:
trainer = Trainer(
    model=model,  # 学習させるモデル
    args=training_args,  # 訓練の引数
    train_dataset=tokenized_data['train'],  # 訓練データセット
    eval_dataset=tokenized_data['val'],  # 評価データセット
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]  # 早期終了のコールバック
)

train_result = trainer.train()  # モデルの学習を開始